In [75]:
import pandas as pd
data = pd.read_json('C:\study\研一上\作业\自然语言处理\data\data2\_valid.json',lines=True)

In [76]:
import pandas as pd

In [77]:
df = pd.read_json('C:\study\研一上\作业\自然语言处理\data\data2\_valid.json',lines=True,orient='records')
df = df[pd.notnull(df['label'])]

In [78]:
print(df.head(11))

In [79]:
with open('C:\study\研一上\作业\自然语言处理\data\data2\label_list.txt', 'r') as f:
    lines = f.readlines()
    labels = []
    for line in lines:
        labels.append(line.strip())
    f.close()

In [80]:
labels

['Contact',
 'Contact_Address_Book',
 'Contact_City',
 'Contact_E_Mail_Address',
 'Contact_Password',
 'Contact_Phone_Number',
 'Contact_Postal_Address',
 'Contact_ZIP',
 'Demographic',
 'Demographic_Age',
 'Demographic_Gender',
 'Facebook_SSO',
 'Identifier',
 'Identifier_Ad_ID',
 'Identifier_Cookie_or_similar_Tech',
 'Identifier_Device_ID',
 'Identifier_IMEI',
 'Identifier_IMSI',
 'Identifier_IP_Address',
 'Identifier_MAC',
 'Identifier_Mobile_Carrier',
 'Identifier_SIM_Serial',
 'Identifier_SSID_BSSID',
 'Location',
 'Location_Bluetooth',
 'Location_Cell_Tower',
 'Location_GPS',
 'Location_IP_Address',
 'Location_WiFi',
 'SSO',
 'No_Mentioned']

In [81]:
import re

In [82]:
REPLACE_BY_SPACE_RE = re.compile('[/{}\[\]\|@,;]')#括号直接去掉，而不是分成两个词
BAD_SYMBOLS_RE = re.compile('[^a-z #+_]')# 不显示数字了

In [83]:
def clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    #text = BeautifulSoup(text, "lxml").text  # HTML decoding
    text = text.lower()  # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text)  # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text)  # delete symbols which are in BAD_SYMBOLS_RE from text
    # text = ' '.join(word for word in text.split() if word not in STOPWORDS)  # delete stopwors from text
    return text

df['text'] = df['text'].apply(clean_text)

In [84]:
df.iloc[0]['text']

'skip to main content home'

In [85]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
import pandas as pd

In [86]:
labels = df['label'].values
labels_dic = {}
for label_list in labels:
    for label in label_list:
        if label not in labels_dic:
            labels_dic[label] = 1
        else:
            labels_dic[label] += 1
labels_dic

{'No_Mentioned': 2201,
 'Contact_E_Mail_Address': 159,
 'Contact_Phone_Number': 118,
 'Contact_Postal_Address': 58,
 'Contact_ZIP': 23,
 'Demographic_Age': 61,
 'Demographic_Gender': 37,
 'Facebook_SSO': 29,
 'Identifier_Ad_ID': 51,
 'Identifier_IP_Address': 107,
 'Identifier_Mobile_Carrier': 17,
 'Location': 153,
 'Location_Cell_Tower': 23,
 'Location_GPS': 30,
 'Location_Bluetooth': 12,
 'Location_WiFi': 17,
 'Contact_Address_Book': 39,
 'Contact_City': 16,
 'Identifier_Cookie_or_similar_Tech': 148,
 'Identifier_MAC': 21,
 'Demographic': 26,
 'SSO': 40,
 'Contact': 40,
 'Identifier_Device_ID': 68,
 'Contact_Password': 35,
 'Location_IP_Address': 21,
 'Identifier_SIM_Serial': 4,
 'Identifier_IMEI': 11,
 'Identifier': 21,
 'Identifier_IMSI': 2,
 'Identifier_SSID_BSSID': 1}

In [87]:
X_train = list(df['text'])
X_train

['skip to main content home',
 'thank you for using our services this privacy policy went into effect on january   ',
 'the weather company  an ibm business  sometimes  we or us is committed to protecting your privacy this privacy policy is important and we encourage you to carefully read it',
 'this privacy policy discloses how we collect  use and share information that we gather about you on the weathercom website the site and our software applications  mobile applications and other websites and services that reference or link to this privacy policy together  the services this privacy policy also describes the choices you have regarding our use of and your ability to correct the information the services do not include websites  mobile applications or other services that link to another privacy policy our truste certification how we collect and use personal data and other information data collection technologies and consumer choice how we share or disclose your information other limit

In [88]:
y_train = df['label']
y_train 

0       [No_Mentioned]
1       [No_Mentioned]
2       [No_Mentioned]
3       [No_Mentioned]
4       [No_Mentioned]
             ...      
2812    [No_Mentioned]
2813    [No_Mentioned]
2814    [No_Mentioned]
2815    [No_Mentioned]
2816    [No_Mentioned]
Name: label, Length: 2817, dtype: object

In [89]:
y_train = list(y_train)

In [90]:
vf = pd.read_json('C:\study\研一上\作业\自然语言处理\data\data2\_valid.json',lines=True)
vf = vf[pd.notnull(vf['label'])]

In [91]:
print(vf.head(5))

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              text  \
0                                                                                                 

In [92]:
X_val = list(vf['text'])
X_val

['Skip to main content Home',
 'Thank you for using our services. This privacy policy went into effect on January 1, 2017.',
 'The Weather Company, an IBM Business, (sometimes, "we" or "us") is committed to protecting your privacy. This Privacy Policy is important and we encourage you to carefully read it.',
 'This Privacy Policy discloses how we collect, use and share information that we gather about you on the weather.com® website (the "Site") and our software applications, mobile applications and other websites and services that reference or link to this Privacy Policy (together, the "Services"). This Privacy Policy also describes the choices you have regarding our use of and your ability to correct the information. The Services do not include websites, mobile applications or other services that link to another privacy policy. Our TRUSTe Certification How We Collect and Use Personal Data and Other Information Data Collection Technologies and Consumer Choice How We Share or Disclose 

In [93]:
y_val = vf['label']
y_val

0       [No_Mentioned]
1       [No_Mentioned]
2       [No_Mentioned]
3       [No_Mentioned]
4       [No_Mentioned]
             ...      
2812    [No_Mentioned]
2813    [No_Mentioned]
2814    [No_Mentioned]
2815    [No_Mentioned]
2816    [No_Mentioned]
Name: label, Length: 2817, dtype: object

In [94]:
y_val = list(y_val)

In [95]:
CV = CountVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2), token_pattern='(\S+)')
feature = CV.fit_transform(X_train)
print(feature)

  (0, 5640)	1
  (0, 6356)	1
  (0, 3477)	1
  (0, 1629)	1
  (0, 2727)	1
  (0, 5641)	1
  (1, 7271)	1
  (1, 2413)	1
  (1, 6855)	1
  (1, 4399)	1
  (1, 5482)	1
  (1, 6287)	1
  (1, 4811)	1
  (1, 4699)	1
  (1, 3137)	1
  (1, 2100)	1
  (1, 4053)	1
  (1, 3257)	1
  (1, 7304)	1
  (1, 2455)	1
  (1, 6861)	1
  (1, 4440)	1
  (1, 5513)	1
  (1, 6303)	1
  (1, 4820)	1
  :	:
  (2815, 4010)	1
  (2815, 5790)	1
  (2815, 5791)	1
  (2815, 5410)	1
  (2815, 1703)	1
  (2815, 3783)	1
  (2815, 4726)	1
  (2815, 2733)	1
  (2816, 5977)	1
  (2816, 3167)	1
  (2816, 466)	1
  (2816, 7179)	1
  (2816, 7208)	1
  (2816, 1)	1
  (2816, 5134)	1
  (2816, 3168)	1
  (2816, 55)	1
  (2816, 6686)	1
  (2816, 5736)	1
  (2816, 6194)	1
  (2816, 6687)	1
  (2816, 4042)	1
  (2816, 6572)	2
  (2816, 4566)	1
  (2816, 2733)	1


In [96]:
tfidf = TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2), token_pattern='(\S+)')
feature = tfidf.fit_transform(X_train)
print(feature)

  (0, 5640)	0.513762115962374
  (0, 6356)	0.10224951143046258
  (0, 3477)	0.4970609205281202
  (0, 1629)	0.2607749928859355
  (0, 2727)	0.38284312477180293
  (0, 5641)	0.513762115962374
  (1, 7271)	0.09178664692540325
  (1, 2413)	0.10567053397775152
  (1, 6855)	0.18601984047930056
  (1, 4399)	0.11535320414848746
  (1, 5482)	0.12435816924435761
  (1, 6287)	0.1281369798088382
  (1, 4811)	0.14452395386357791
  (1, 4699)	0.15370038573788292
  (1, 3137)	0.2573904779006788
  (1, 2100)	0.3243708750053188
  (1, 4053)	0.12266201351998317
  (1, 3257)	0.37095955502555095
  (1, 7304)	0.286258171419267
  (1, 2455)	0.393254057424492
  (1, 6861)	0.286258171419267
  (1, 4440)	0.18214173954066457
  (1, 5513)	0.3298739648206272
  (1, 6303)	0.20384816137705544
  (1, 4820)	0.17291080833858535
  :	:
  (2815, 4010)	0.16272374230735645
  (2815, 5790)	0.27402435136782244
  (2815, 5791)	0.27935492169459375
  (2815, 5410)	0.25464288978523564
  (2815, 1703)	0.17201733136129113
  (2815, 3783)	0.22546587613570862


In [97]:
y_train

[['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['No_Mentioned'],
 ['Contact_E_Mail_Address',
  'Contact_Phone_Number',
  'Contact_Postal_Address',
  'Contact_ZIP',
  'Demographic_Age',
  'Demographic_Gender',
  'Facebook_SSO',
  'Identifier_Ad_ID',
  'Identifier_IP_Address',
  'Identifier_Mobile_Carrier',
  'Location',
  'Location_Cell_Tower',
  'Location_GPS'],
 ['No_Mentioned'],
 ['Contact_E_Mail_Address',
  'Contact_Phone_Number',
  'Contact_Postal_Address',
  'Contact_ZIP',
  'Identifier_Ad_ID',
  'Identifier_IP_Address',
  'Identifier_Mobile_Carrier',
  'Location',
  'Location_Cell_Tower',
  'Location_GPS'],
 ['Contact_E_Mail_Address',
  'Contact_Phone_Number',
  'Contact_Postal_Address',
  'Contact_ZIP',
  'Identifier_Ad_ID',
  'Identifier_IP_Address',
  'Identifier_Mobile_Carrier',
  'Location',
  'Location_Cell_Tower',
  'Location_GPS'],
 ['No_Mentioned'],
 ['Location'],
 ['Location'],
 ['Contact_E_Mail_Address

In [98]:
mlb = MultiLabelBinarizer(classes=sorted(labels_dic.keys()))
y_train = mlb.fit_transform(y_train)
y_val = mlb.fit_transform(y_val)

In [99]:
y_train

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [100]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

def evaluation(y_val, predicted):
    accuracy = accuracy_score(y_val, predicted)
    f1_score_macro = f1_score(y_val, predicted, average='macro')
    print("accuracy:", accuracy)
    print('f1_score_macro',f1_score_macro)

In [101]:
pipeline1 = Pipeline([
    ('tfidf', TfidfVectorizer(min_df=5, max_df=0.9, ngram_range=(1,2), token_pattern='(\S+)')),
    ('lsvc', OneVsRestClassifier(LinearSVC()))
])
pipeline1.fit(X_train,y_train)
predicted = pipeline1.predict(X_val)
evaluation(y_val, predicted)

accuracy: 0.9265175718849841
f1_score_macro 0.8061107064429117


In [102]:
test_df = pd.read_csv('ceshitest.txt', sep='\0', header=None)
test_df.columns = ['text']

In [103]:
test_df

,text
0,"More All All Live Stations Artists Songs Podcasts Share Privacy and Cookie Notice November 10, 2016 Versión en Español UPDATED AS OF: November 10, 2016 iHeartMedia, Inc. (""iHeartMedia"", ""we"", ""us"", or ""our"") is committed to maintaining your confidence and trust as it relates to the privacy of your information. Please read below and learn how we collect, protect, share and use your information as part of our technology platforms, including, without limitation, our websites, web pages, interactive features, applications, Twitter and Facebook pages, and Mobile Application (""Platforms"")."
1,"QUICK GUIDE TO CONTENTS INFORMATION COLLECTED ON OUR PLATFORMS HOW WE USE THE INFORMATION WE COLLECT SHARING OF INFORMATION INFORMATION WE RECEIVE FROM THIRD PARTIES YOUR PRIVACY RIGHTS, CHOICE AND ACCESS ADVERTISING/BEHAVIORAL TARGETING; HOW TO OPT-OUT CHILDREN SECURITY OF YOUR INFORMATION OTHER SITES CONSENT TO PROCESSING AND TRANSFER OF INFORMATION CHANGES CONTACT US"
2,"1. INFORMATION COLLECTED ON OUR PLATFORMS Information You Provide To Us We or our service providers may collect Personal Information (information that can be used to identify you as an individual) such as your name, email, telephone number, home address, demographic information (such as zip code, age and gender), location information, or payment information (such as account or credit card number). The types of Personal Information collected may vary depending on your use of the features of the Platforms. For example, your credit card number and other payment related information may be collected in connection with your purchase of products and/or services through the Platforms. We post customer testimonials/video testimonials on our Platforms which may contain personally identifiable information. If we want to post a customer's name along with their testimonial, we obtain the customer's consent via email prior to posting the testimonial."
3,"Information Collected Automatically Usage Information. Whenever you visit or interact with the Platforms, we, as well as any third-party advertisers and/or service providers, may use a variety of technologies that automatically or passively collect information about how the Platforms are accessed and used (""Usage Information""). Usage Information may include Device Identifiers (as defined below), browser type, device type, operating system, application version, the page served, the time, the preceding page views, and your use of features or applications on the Platforms, such as song history, interaction with friends and group activities. This information helps us keep our Platforms fresh and interesting to our visitors and allows us to tailor content to a visitor's interests."
4,"Device Identifier. We or our service providers automatically collect your IP address or other unique identifier (""Device Identifier"") for the Device (computer, mobile phone, tablet or other device) you use to access the Platforms. A Device Identifier is a number such as a mobile advertising identifier (Android AAID or Apple IDFA) that is assigned to your Device when you access a website or its servers, and our computers identify your Device by its Device Identifier. We may use a Device Identifier to, among other things, administer the Platforms, help diagnose problems with our servers, analyze trends, provide attribution metrics to our advertisers and partners, track users' web page movements, help identify you and your shopping cart, and gather broad demographic information for aggregate use."
...,...
2535,We retain the information we collect as described in this Privacy Policy for as long as necessary to fulfill our business and/or operational purposes or comply with a legal request. We may also choose to anonymize parts of the information we collect so that it can no longer be identifiable or attributable to any person if we want to keep it for longer periods of time.
2536,Transfer of Data Outside Your Territory.
2537,The Software & Ser

In [104]:
testSet = list(test_df['text'])

In [105]:
test_predicted_labels = pipeline1.predict(testSet)

In [106]:
test_predicted_labels

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [107]:
test_original_labels = mlb.inverse_transform(test_predicted_labels)

In [108]:
test_original_labels

[('No_Mentioned',),
 ('No_Mentioned',),
 ('Contact_E_Mail_Address',),
 (),
 ('Identifier_Device_ID',),
 ('Location',),
 ('Identifier_Cookie_or_similar_Tech',),
 ('Identifier_Cookie_or_similar_Tech',),
 ('Identifier_Cookie_or_similar_Tech',),
 (),
 ('Identifier_Cookie_or_similar_Tech',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('Contact_E_Mail_Address', 'SSO'),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('Location',),
 (),
 (),
 ('No_Mentioned',),
 ('No_Mentioned',),
 (),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 (),
 ('Contact_E_Mail_Address',),
 ('No_Mentioned',),
 (),
 ('Contact_E_Mail_Address',),
 ('No_Mentioned',),
 ('Identifier_IP_Address',),
 ('No_Mentioned',),
 ('No_Mentioned',),
 ('Identifier_Cookie_or_similar_Tech',),
 ('No_Mentioned',),
 ('No

In [111]:
with open("test_labels-2.txt", "w") as file:
    for tol in test_original_labels:
        line = ','.join(tol)
        if not line.strip():
            line = "No_Mentioned"
        file.write(line + '\n')